In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Output, Button
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output, Javascript
from base64 import b64encode
%matplotlib inline

##### DEFINE FUNCTIONS #####

def scaling_factor(NA,n1,n2,lam_0):
    z = np.arange(lam_0,100,0.1)
    n2overn1 = np.divide(n2,n1)
    
    if n2overn1 < 1: eps = np.multiply(-1,np.divide(np.divide(lam_0,4),(np.multiply(z,n2))))
    else: eps = np.divide(np.divide(lam_0,4),(np.multiply(z,n2)))
    eps_term = np.multiply(eps, np.subtract(2,eps))    
    
    m = np.emath.sqrt(np.subtract(np.power(n2,2),np.power(n1,2)))
    
    sf_univ = np.multiply(np.divide(n2,n1),
                          np.divide(1-eps+np.divide(m,n1)*np.emath.sqrt(eps_term),
                                    1-np.multiply(np.divide(n2,n1)**2,eps_term)))
    sf_crit = np.divide(n1-np.emath.sqrt(np.power(n1,2)-np.power(NA,2)),
                        n2-np.emath.sqrt(np.power(n2,2)-np.power(NA,2)))
    
    sf = np.zeros(len(z))
    for i in range(len(sf)):
        if n2overn1 < 1: sf[i] = np.max([np.real(sf_univ[i]),np.real(sf_crit)])
        elif n2overn1 > 1:sf[i] = np.min([np.real(sf_univ[i]),np.real(sf_crit)])
        else: sf[i]=1
    return z,sf,sf_crit,n2overn1

def plot_scaling_factor(NA,n1,n2,lam_0):
    z,sf,sf_crit,n2overn1=scaling_factor(NA,n1,n2,lam_0)
    plt.figure(figsize=(10,6))
    if n2overn1 > 1:   plt.ylim([n2overn1*0.9,1.1*np.real(sf_crit)])
    elif n2overn1 < 1: plt.ylim([0.8*np.real(sf_crit),n2overn1*1.1])
    else: plt.ylim(0.5,1.5)
    plt.xlim([0,100])
    plt.xlabel(r'Depth ($\mu$m)')
    plt.ylabel(r'Scaling factor $\zeta$')
    plt.grid(ls=':')
    plt.plot(z,sf)

def csv_file(x,y):
    # add column names
    string = 'Depth (microns),Scaling Factor\n'
    for i in range(len(x)): #run over all data points and add to file
        string+=(str(x[i])+','+str(y[i])+'\n')
    return string

def trigger_download(text, filename, kind='text/json'):
    # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
    content_b64 = b64encode(text.encode()).decode()
    data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download', '{filename}');
        a.setAttribute('href', '{data_url}');
        a.click()
    """
    with download_output:
        clear_output()
        display(HTML(f'<script>{js_code}</script>'))

def download_csv(e=None):
    NA = widget.widget.children[0].value
    n1 = widget.widget.children[1].value
    n2 = widget.widget.children[2].value
    lam_0 = widget.widget.children[3].value
    z,sf,_,_=scaling_factor(NA,n1,n2,lam_0)
    trigger_download(csv_file(z,sf), 'sf_vs_depth.csv', kind='text/plain')
    
def window_open_button(url):
    with out:
        display(Javascript(f'window.open("{url.tooltip}");'))


##### START CODE #####
        
print("Scaling factor calculator \n\nAdjust the sliders to set the parameters of the plot")    
  
style = {'description_width': 'initial'}
slider_width = '40%'
widget = interact(plot_scaling_factor, NA = widgets.FloatSlider(value=0.85, description='Numerical aperture ----------', 
                                        style=style, layout = Layout(width=slider_width),
                                               min=0.1,
                                               max=1.5,
                                               step=0.05),
        n1 = widgets.FloatSlider(value=1, description='Immersion refractive index $n_1$', 
                                        style=style,layout = Layout(width=slider_width),
                                               min=1,
                                               max=1.52,
                                               step=0.01),
        n2 = widgets.FloatSlider(value=1.33,description='Sample refractive index $n_2$ --', 
                                        style=style,layout = Layout(width=slider_width),
                                               min=1,
                                               max=1.52,
                                               step=0.01),
        lam_0 = widgets.FloatSlider(value=0.52,description=r'Wavelength $\lambda_0$ ($\mu$m) ---------', 
                                        style=style,layout = Layout(width=slider_width),
                                               min=0.2,
                                               max=0.9,
                                               step=0.02))

plt.show()

download_output = Output()
display(download_output)
out = Output()
display(out)

btn = Button(description='Download as .csv')
btn2 = Button(description="Source", tooltip='https://github.com/hoogenboom-group/SF')

btn.on_click(download_csv)
btn2.on_click(window_open_button)

display(btn,btn2)


Scaling factor calculator 

Adjust the sliders to set the parameters of the plot


interactive(children=(FloatSlider(value=0.85, description='Numerical aperture ----------', layout=Layout(width…

Output()

Output()

Button(description='Download as .csv', style=ButtonStyle())

Button(description='Source', style=ButtonStyle(), tooltip='https://github.com/hoogenboom-group/SF')